<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/2_waymo_official_tutorial/gp-waymo_official_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waymo 공식 튜토리얼 실습
https://github.com/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial_motion.ipynb

# 패키지 설치

## waymo_open_dataset 패키지 설치하기

In [96]:
%%capture
!rm -rf waymo-od > /dev/null # 레포가 이미 존재하면 지움
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git checkout remotes/origin/master

In [97]:
%%capture
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset-tf-2-6-0

## Read one frame
하나의 프레임을 읽어보자. 파일이 어떻게 생겼고, 어떻게 읽을 수 있는지 알아본다

In [98]:
!ls -al ./waymo-od/tutorial/frames

-rw-r--r-- 1 root root 8577921 Mar 28 11:29 ./waymo-od/tutorial/frames


In [99]:
!xxd ./waymo-od/tutorial/frames | head

00000000: ae5a 4100 0000 0000 e0a5 4a4b 0aa8 150a  .ZA.......JK....
00000010: 2631 3130 3034 3638 3537 3339 3731 3435  &110046857397145
00000020: 3030 3232 305f 3233 3030 5f30 3030 5f32  00220_2300_000_2
00000030: 3332 305f 3030 3012 ee01 0801 11dd ef9b  320_000.........
00000040: bf1c 0fa0 4011 ddef 9bbf 1c0f a040 11d8  ....@........@..
00000050: 5892 7f42 5d8d 4011 e81f 42d4 9308 8440  X..B].@...B....@
00000060: 118f b145 4fb9 8ba0 3f11 41c3 45a9 0592  ...EO...?.A.E...
00000070: d4bf 11ba 4ac5 6e55 fe49 3f11 85b3 7881  ....J.nU.I?...x.
00000080: 2a81 44bf 1100 0000 0000 0000 001a 9001  *.D.............
00000090: 0935 29b1 f11e ffef 3f09 0c0a 38a1 558c  .5).....?...8.U.


In [100]:
import tensorflow.compat.v1 as tf

# Eager Execution 모드 활성화 -> 디버깅 용이 및 코드 실행속도 향상
tf.enable_eager_execution()

import numpy as np
from waymo_open_dataset import dataset_pb2 # pb : protocol buffers

FILENAME = '/content/waymo-od/tutorial/frames' # git repo 내에 있음
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
for data in dataset:
    frame = dataset_pb2.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break

In [101]:
from waymo_open_dataset.utils import  frame_utils

(range_images, camera_projections, _, range_image_top_pose)\
  = frame_utils.parse_range_image_and_camera_projection(frame)

# 데이터셋 로드

## Google Drive 연동
일단 테스트용도로 validation 5개를 drive에 업로드 했다. 이를 가져온다.

In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
!cp -r /content/drive/MyDrive/waymo-od-dataset/for_tutorial ./

In [94]:
!ls for_tutorial/

validation.tfrecord-00000-of-00150  validation.tfrecord-00003-of-00150
validation.tfrecord-00001-of-00150  validation.tfrecord-00004-of-00150
validation.tfrecord-00002-of-00150


In [95]:
!cd /content/drive/MyDrive/waymo-od-dataset/ &\
 ls -al ./for_tutorial/validation.tfrecord-00000-of-00150

-rw------- 1 root root 254565845 Mar 28 11:29 ./for_tutorial/validation.tfrecord-00000-of-00150


## python 코드로 데이터셋 읽기

In [107]:
DATASET_DIR = '/content/drive/MyDrive/waymo-od-dataset'
FILENAME = DATASET_DIR + '/for_tutorial/validation.tfrecord-00000-of-00150'

In [108]:
import tensorflow as tf

dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [109]:
for num_data, data in enumerate(dataset):
    pass

num_data

269

하나의 tfrecord 파일 내부에는 269개의 데이터가 있는 것을 알 수 있다.

### 데이터셋 구조 파일(proto)을 이용해서 로드하기

tfrecord 데이터셋은 압축된 형태로 존재한다. TFRecordDataset을 이용해서 tfrecord 데이터셋을 읽을 수 있으나, 압축을 해제할 수 있도록 구조 정의가 필요하다.

직접 정의할 수도 있으나, waymo-open-dataset 라이브러리를 이용하면 구조(proto)를 가져올 수 있다.

In [111]:
type(data)

tensorflow.python.framework.ops.EagerTensor

In [112]:
from waymo_open_dataset.protos import scenario_pb2

scenario = scenario_pb2.Scenario()
scenario.ParseFromString(data.numpy())

435428

In [114]:
scenario.scenario_id

'c12dd9b235799eb8'

구조 정의가 되었으니, 이를 기반으로 dataset을 해석할 수 있다. 한 개만 뽑아서 풀어본다.

UnicodeDecodeError: ignored

In [76]:
!ls waymo-od/tutorial/frames_with_maps.tfrecord

waymo-od/tutorial/frames_with_maps.tfrecord


In [ ]:
# tutorial/2_waymo_official_tutorial/gp-waymo_official_tutorial.ipynb